# information

> This module contains code for the information tab of the application

In [1]:
#| default_exp selection

In [2]:
#| hide
from nbdev.showdoc import *

These are the relevant library calls to build out the UI and the calls to the model in order to set up the manuscript selection buttons.

In [3]:
#| hide
# This library is only for testing the layout. It does not need to be called extraneously
from dash import Dash

In [4]:
#| export
from dash import dcc, html
from glyptodon.manuscriptFiles import updateMetadata

## labeledInput

For this tab, we want ```Input``` elements. Unfortunately, this will be hard to read unless the ```Input``` has a ```Label```. For this reason, the ```labeledInput``` tab takes in a single value for the label text and placeholder value for ```Input``` and returns a ```Div``` object with both a ```Label``` and an ```Input``` for children.

In [5]:
#| export
def labeledInput(label, identity = None):
    lab = html.Label(children=label)
    inp = dcc.Input(placeholder=label, id=identity)

    return html.Div([lab, inp])

In [ ]:
#| hide
app = Dash(__name__)

app.layout = html.Div(
    [
        labeledInput(label="test", placeholder="test"),
        labeledInput(label="testy", placeholder="testy"),
    ]
)

if __name__ == "__main__":
    app.run(debug=True)

## createInputObjects

It would be ideal to have a function which would give us ```Input``` objects for metadata manipulation and return them alongside a useful key to quickly set their values. This function utilizes ```labeledInput``` to make that as simple as possible. It returns:
- A ```list``` containing the metadata labels that can be used to quickly access a dictionary in sequence
- A ```list``` containing the ```Input``` objects

In [15]:
#| export
def createInputObjects():
    inputObjects = []
    metadata = ["Work", "Author", "Language", "Country", "City", "Institution"]

    for data in metadata:
        inputObjects.append(labeledInput(label = data, identity = data.lower()))

    return metadata, inputObjects

In [ ]:
#| hide
app = Dash(__name__)

app.layout = html.Div(createInputObjects()[1])

if __name__ == "__main__":
    app.run(debug=True)

## createCenturiesSlider

In addition to the ```Input``` objects which contain important metadata, we also want the user to be able to edit data regarding the century of the manuscripts so that the time a manuscript is written is still a part of the objects data. This function uses the [```RangeSlider```](https://dash.plotly.com/dash-core-components/rangeslider) object. Because of the smaller size this object needs to have in a window, the ```marks``` of ```RangeSlider``` are not displayed through the object. Rather, an attached label is updated in the main application. Additionally, this object only allows users to input centuries from the first century AD/CE forward and not earlier. This is not an inherently necessary design decision, but it simplifies considerably the display of the widget. Should this be a poor choice, it will be revisited later. This function returns:
- A ```list``` containing the possible century options indexed to their easy numerical value (i.e. "First century" indexed to 1) and nothing in the zero index
- A ```Div``` object containing a ```RangeSlider``` object with 20 possible options from 1 to 20 and a label for it

In [7]:
#| export
def createCenturiesSlider():
    centuries = [
        None,
        "1st",
        "2nd",
        "3rd",
    ] + [f"{i}th" for i in range(4, 21)]

    centuriesSlider = dcc.RangeSlider(
        min=1,
        max=20,
        step=1,
        value=[1, 20],
        id="centuries-slider",
        marks=None,
    )

    return centuries, html.Div([html.Label(children="Centuries"),centuriesSlider])

In [ ]:
#| hide
app = Dash(__name__)

app.layout = html.Div(createCenturiesSlider()[1])

if __name__ == "__main__":
    app.run(debug=True)

## createUploadObjects

In order for users to fully utilize this application, we want them to be able to upload files through the application instead of placing them into file folders. Using the available libraries, the [```Upload```](https://dash.plotly.com/dash-core-components/upload) object is the best object for this task. When files are uploaded in this object, they are stored as binary string, so calls from ```manuscriptFiles``` are necessary to take these from memory and into storage. This function returns:
- An ```Upload``` object that is for images
- An ```Upload``` object that is for manuscripts

In [8]:
#| export
def createUploadObjects():
    uploadImages = dcc.Upload(
        children=html.Button("Upload Images"),
        multiple=True,
        id="upload-images",
    )

    uploadManuscripts = dcc.Upload(
        children=html.Button("Upload Manuscripts"),
        multiple=True,
        id="upload-manuscripts",
    )

    return uploadImages, uploadManuscripts

In [ ]:
#| hide
app = Dash(__name__)

app.layout = html.Div(createUploadObjects())

if __name__ == "__main__":
    app.run(debug=True)

## createInformationInfo

Just as the ```selection``` module has an info pane to display info on how to use that tab, this module also needs an info pane. As before, it uses the ```Markdown``` object to create the relevant information.

In [9]:
#| export
def createInformationInfo():
    return dcc.Markdown(
        """
    # Information
    
    This menu allows you to modify the information about a manuscript and upload images and transcripts for a manuscript.
    
    Once you have made your changes and uploaded the relevant files, click the Save and Continue button to save these changes and move to the next tab.
    """
    )

In [ ]:
#| hide
createInformationInfo()

## createSaveNContinue

This function returns a ```Button``` that will be linked to a [callback](https://dash.plotly.com/basic-callbacks) which saves the data from the various objects in this tab and then moves onto the next one.

In [10]:
#| export
def createSaveNContinue():
    return html.Button("Save and Continue", id="save-and-continue")

## createInformationLayout

As before, we need a layout that fits all of these widgets together. Because this will be used in another program, the information we want to preserve from the function calls in this module will also be returned. This function returns:
- ```centuries```
- ```metadata```
- A ```Div``` object containg the tab layout to be put into a ```Tab``` object

In [16]:
#| export
def createInformationLayout():
    metadata, inputObjects = createInputObjects()
    centuries, centuriesSlider = createCenturiesSlider()
    uploadImages, uploadManuscripts = createUploadObjects()
    saveNContinue = createSaveNContinue()
    
    layout = html.Div(
        [
            createInformationInfo(),
            html.Br(), # This is a temporary measure to make it more readable, true for following Br calls
            html.Div(inputObjects),
            html.Br(),
            html.Div(
                [
                    uploadImages,
                    uploadManuscripts,
                ]
            ),
            html.Br(),
            saveNContinue,
        ]
    )
    
    return metadata, centuries, layout

In [17]:
#| hide
app = Dash(__name__)

app.layout = createInformationLayout()[2]

if __name__ == "__main__":
    app.run(debug=True)

In [ ]:
#| hide
import nbdev

nbdev.nbdev_export()